# Looking into the Lasso technique for regression, trying to predict the 3rd grade based on a series of features

## First looking for the Portuguese students

In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

students_portuguese = pd.read_csv('../data/student-por.csv', sep = ';')

students_portuguese

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13
5,GP,M,16,U,LE3,T,4,3,services,other,...,5,4,2,1,2,5,6,12,12,13
6,GP,M,16,U,LE3,T,2,2,other,other,...,4,4,4,1,1,3,0,13,12,13
7,GP,F,17,U,GT3,A,4,4,other,teacher,...,4,1,4,1,1,1,2,10,13,13
8,GP,M,15,U,LE3,A,3,2,services,other,...,4,2,2,1,1,1,0,15,16,17
9,GP,M,15,U,GT3,T,3,4,other,other,...,5,5,1,1,1,5,0,12,12,13


The quantitative variables (with the exception of age) and the ordinal variables (with the exception of traveltime) shall be used for the lasso model. The factor variables are explainable via the ordinal variables.

Model '1'

In [2]:
features = students_portuguese.loc[:, ['absences', 'G1', 'G2', 'Medu', 'Fedu', 'studytime', 
                                                'failures', 'famrel', 'freetime',
                                                'goout', 'Dalc', 'Walc', 'health']]
target   = students_portuguese.loc[:, 'G3']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2)


In [3]:
print('Default number of features: {}'.format(features.shape[1]))
print('=' * 100)

lasso_normal = Lasso(max_iter = 10e6)
lasso_normal = lasso_normal.fit(X_train, y_train)

train_score = lasso_normal.score(X_train, y_train)
test_score  = lasso_normal.score(X_test, y_test)
coefficients_used = np.sum(lasso_normal.coef_ != 0)

print("Training score default lasso: {}".format(train_score))
print("Test score default lasso: {}".format(test_score))
print("Number of features used: {}".format(coefficients_used))

coef = np.array(lasso_normal.coef_)
most_relevant = coef.argsort()[-10:]
print("Most relevant columns: {}".format(features.columns[most_relevant[::-1]]))

alpha_values = [0.1, 0.01, 0.001, 2, 5, 10]
for alpha_iter in alpha_values:
    
    lasso = Lasso(alpha = alpha_iter, max_iter = 10e6)
    lasso = lasso.fit(X_train, y_train)

    train_score = lasso.score(X_train, y_train)
    test_score  = lasso.score(X_test, y_test)
    coefficients_used = np.sum(lasso.coef_ != 0)

    print("-" * 100)
    print("Training score alpha = {} lasso: {}".format(alpha_iter, train_score))
    print("Test score alpha = {} lasso: {}".format(alpha_iter, test_score))
    print("Number of features used: {}".format(coefficients_used))

    coef = np.array(lasso.coef_)
    most_relevant = coef.argsort()[-10:]
    print("Most relevant columns: {}".format(features.columns[most_relevant[::-1]]))


Default number of features: 13
Training score default lasso: 0.8421279727526938
Test score default lasso: 0.8116479661181969
Number of features used: 2
Most relevant columns: Index(['G2', 'G1', 'health', 'Walc', 'Dalc', 'goout', 'freetime', 'famrel',
       'failures', 'studytime'],
      dtype='object')
----------------------------------------------------------------------------------------------------
Training score alpha = 0.1 lasso: 0.8552932409091669
Test score alpha = 0.1 lasso: 0.8039754368230476
Number of features used: 4
Most relevant columns: Index(['G2', 'G1', 'absences', 'health', 'Walc', 'Dalc', 'goout', 'famrel',
       'failures', 'studytime'],
      dtype='object')
----------------------------------------------------------------------------------------------------
Training score alpha = 0.01 lasso: 0.8599106075968276
Test score alpha = 0.01 lasso: 0.7959275598251783
Number of features used: 12
Most relevant columns: Index(['G2', 'G1', 'studytime', 'absences', 'Fedu', 'W

## Now looking at the Mathematics students

In [4]:
students_mathematics = pd.read_csv('../data/student-mat.csv', sep = ';')

students_mathematics

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
5,GP,M,16,U,LE3,T,4,3,services,other,...,5,4,2,1,2,5,10,15,15,15
6,GP,M,16,U,LE3,T,2,2,other,other,...,4,4,4,1,1,3,0,12,12,11
7,GP,F,17,U,GT3,A,4,4,other,teacher,...,4,1,4,1,1,1,6,6,5,6
8,GP,M,15,U,LE3,A,3,2,services,other,...,4,2,2,1,1,1,0,16,18,19
9,GP,M,15,U,GT3,T,3,4,other,other,...,5,5,1,1,1,5,0,14,15,15


Model '2'

In [5]:
features = students_mathematics.loc[:, ['absences', 'G1', 'G2', 'Medu', 'Fedu', 'studytime', 
                                                'failures', 'famrel', 'freetime',
                                                'goout', 'Dalc', 'Walc', 'health']]
target   = students_mathematics.loc[:, 'G3']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2)

In [6]:
print('Default number of features: {}'.format(features.shape[1]))
print('=' * 100)

lasso_normal = Lasso(max_iter = 10e6)
lasso_normal = lasso_normal.fit(X_train, y_train)

train_score = lasso_normal.score(X_train, y_train)
test_score  = lasso_normal.score(X_test, y_test)
coefficients_used = np.sum(lasso_normal.coef_ != 0)

print("Training score default lasso: {}".format(train_score))
print("Test score default lasso: {}".format(test_score))
print("Number of features used: {}".format(coefficients_used))

coef = np.array(lasso_normal.coef_)
most_relevant = coef.argsort()[-10:]
print("Most relevant columns: {}".format(features.columns[most_relevant[::-1]]))

alpha_values = [0.1, 0.01, 0.001, 2, 5, 10, 15]
for alpha_iter in alpha_values:
    
    lasso = Lasso(alpha = alpha_iter, max_iter = 10e6)
    lasso = lasso.fit(X_train, y_train)

    train_score = lasso.score(X_train, y_train)
    test_score  = lasso.score(X_test, y_test)
    coefficients_used = np.sum(lasso.coef_ != 0)

    print("-" * 100)
    print("Training score alpha = {} lasso: {}".format(alpha_iter, train_score))
    print("Test score alpha = {} lasso: {}".format(alpha_iter, test_score))
    print("Number of features used: {}".format(coefficients_used))

    coef = np.array(lasso.coef_)
    most_relevant = coef.argsort()[-10:]
    print("Most relevant columns: {}".format(features.columns[most_relevant[::-1]]))

Default number of features: 13
Training score default lasso: 0.80463754322683
Test score default lasso: 0.9139290817097182
Number of features used: 3
Most relevant columns: Index(['G2', 'G1', 'absences', 'health', 'Walc', 'Dalc', 'goout', 'freetime',
       'famrel', 'failures'],
      dtype='object')
----------------------------------------------------------------------------------------------------
Training score alpha = 0.1 lasso: 0.819908022523941
Test score alpha = 0.1 lasso: 0.8864092714402222
Number of features used: 7
Most relevant columns: Index(['G2', 'famrel', 'G1', 'Walc', 'absences', 'health', 'Dalc', 'goout',
       'freetime', 'studytime'],
      dtype='object')
----------------------------------------------------------------------------------------------------
Training score alpha = 0.01 lasso: 0.8238747898758564
Test score alpha = 0.01 lasso: 0.8736179265675106
Number of features used: 13
Most relevant columns: Index(['G2', 'famrel', 'Walc', 'G1', 'health', 'Medu', 'ab

## Looking at the inner joined data set, the one containing the students that took both the Mathematics and the Portuguese classes

In [7]:
students_combined = pd.read_csv('../data/students-combined.csv', sep = ';')
students_combined = students_combined.drop(columns = 'Unnamed: 0')

students_combined

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,health,absences_x,G1_x,G2_x,G3_x,paid_y,absences_y,G1_y,G2_y,G3_y
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,3,4,0,11,11,no,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,3,2,9,11,11,no,4,5,5,6
2,GP,F,15,U,GT3,T,4,2,health,services,...,5,0,14,14,14,yes,2,15,14,15
3,GP,F,16,U,GT3,T,3,3,other,other,...,5,0,11,13,13,yes,4,6,10,10
4,GP,M,16,U,LE3,T,4,3,services,other,...,5,6,12,12,13,yes,10,15,15,15
5,GP,M,16,U,LE3,T,2,2,other,other,...,3,0,13,12,13,no,0,12,12,11
6,GP,F,17,U,GT3,A,4,4,other,teacher,...,1,2,10,13,13,no,6,6,5,6
7,GP,M,15,U,LE3,A,3,2,services,other,...,1,0,15,16,17,yes,0,16,18,19
8,GP,M,15,U,GT3,T,3,4,other,other,...,5,0,12,12,13,yes,0,14,15,15
9,GP,F,15,U,GT3,T,4,4,teacher,health,...,2,2,14,14,14,yes,0,10,8,9


Trying to predict the final grade for both Portuguese and Mathematics using the prior information

Model '3'

In [8]:
features = students_combined.loc[:, ['absences_x', 'G1_x', 'G2_x', 'Medu', 'Fedu', 'studytime', 
                                                'failures', 'famrel', 'freetime',
                                                'goout', 'Dalc', 'Walc', 'health', 'absences_y', 'G1_y', 'G2_y']]
target   = students_combined.loc[:, ['G3_x', 'G3_y']]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2)

In [9]:
print('Default number of features: {}'.format(2 * features.shape[1]))
print('{} features for the first predicted value and {} for the second one'.format(features.shape[1], features.shape[1]))
print('=' * 100)

lasso_normal = Lasso(max_iter = 10e6)
lasso_normal = lasso_normal.fit(X_train, y_train)

train_score = lasso_normal.score(X_train, y_train)
test_score  = lasso_normal.score(X_test, y_test)
coefficients_used = np.sum(lasso_normal.coef_ != 0)

print("Training score default lasso: {}".format(train_score))
print("Test score default lasso: {}".format(test_score))
print("Number of features used: {}".format(coefficients_used))

coef1, coef2 = np.array(lasso_normal.coef_)
most_relevant1 = coef1.argsort()[-10:]
most_relevant2 = coef2.argsort()[-10:]
print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))

alpha_values = [0.1, 0.01, 0.001, 2, 5, 10, 15]
for alpha_iter in alpha_values:
    
    lasso = Lasso(alpha = alpha_iter, max_iter = 10e6)
    lasso = lasso.fit(X_train, y_train)

    train_score = lasso.score(X_train, y_train)
    test_score  = lasso.score(X_test, y_test)
    coefficients_used = np.sum(lasso.coef_ != 0)

    print("-" * 100)
    print("Training score alpha = {} lasso: {}".format(alpha_iter, train_score))
    print("Test score alpha = {} lasso: {}".format(alpha_iter, test_score))
    print("Number of features used: {}".format(coefficients_used))

    coef1, coef2 = np.array(lasso.coef_)
    most_relevant1 = coef1.argsort()[-10:]
    most_relevant2 = coef2.argsort()[-10:]
    print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
    print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))

Default number of features: 32
16 features for the first predicted value and 16 for the second one
Training score default lasso: 0.8020543425032639
Test score default lasso: 0.8706809463920581
Number of features used: 5
Most relevant columns for Portuguese: Index(['G2_x', 'G1_x', 'G2_y', 'G1_y', 'absences_y', 'health', 'Walc', 'Dalc',
       'goout', 'freetime'],
      dtype='object')
Most relevant columns for Mathematics: Index(['G2_y', 'G1_y', 'absences_y', 'health', 'Walc', 'Dalc', 'goout',
       'freetime', 'famrel', 'failures'],
      dtype='object')
----------------------------------------------------------------------------------------------------
Training score alpha = 0.1 lasso: 0.8230953384426963
Test score alpha = 0.1 lasso: 0.8789156535356152
Number of features used: 15
Most relevant columns for Portuguese: Index(['G2_x', 'G1_x', 'absences_x', 'G2_y', 'G1_y', 'health', 'Walc', 'goout',
       'freetime', 'famrel'],
      dtype='object')
Most relevant columns for Mathematic

Removing the second grade from the analysis

Model '4'

In [10]:
features = students_combined.loc[:, ['absences_x', 'G1_x', 'Medu', 'Fedu', 'studytime', 
                                                'failures', 'famrel', 'freetime',
                                                'goout', 'Dalc', 'Walc', 'health', 'absences_y', 'G1_y']]
target   = students_combined.loc[:, ['G3_x', 'G3_y']]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2)

In [11]:
print('Default number of features: {}'.format(2 * features.shape[1]))
print('{} features for the first predicted value and {} for the second one'.format(features.shape[1], features.shape[1]))
print('=' * 100)

lasso_normal = Lasso(max_iter = 10e6)
lasso_normal = lasso_normal.fit(X_train, y_train)

train_score = lasso_normal.score(X_train, y_train)
test_score  = lasso_normal.score(X_test, y_test)
coefficients_used = np.sum(lasso_normal.coef_ != 0)

print("Training score default lasso: {}".format(train_score))
print("Test score default lasso: {}".format(test_score))
print("Number of features used: {}".format(coefficients_used))

coef1, coef2 = np.array(lasso_normal.coef_)
most_relevant1 = coef1.argsort()[-10:]
most_relevant2 = coef2.argsort()[-10:]
print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))

alpha_values = [0.1, 0.01, 0.001, 2, 5, 10, 15]
for alpha_iter in alpha_values:
    
    lasso = Lasso(alpha = alpha_iter, max_iter = 10e6)
    lasso = lasso.fit(X_train, y_train)

    train_score = lasso.score(X_train, y_train)
    test_score  = lasso.score(X_test, y_test)
    coefficients_used = np.sum(lasso.coef_ != 0)

    print("-" * 100)
    print("Training score alpha = {} lasso: {}".format(alpha_iter, train_score))
    print("Test score alpha = {} lasso: {}".format(alpha_iter, test_score))
    print("Number of features used: {}".format(coefficients_used))

    coef1, coef2 = np.array(lasso.coef_)
    most_relevant1 = coef1.argsort()[-10:]
    most_relevant2 = coef2.argsort()[-10:]
    print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
    print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))

Default number of features: 28
14 features for the first predicted value and 14 for the second one
Training score default lasso: 0.6585947271185931
Test score default lasso: 0.712618927757402
Number of features used: 6
Most relevant columns for Portuguese: Index(['G1_x', 'G1_y', 'health', 'Walc', 'Dalc', 'goout', 'freetime', 'famrel',
       'failures', 'studytime'],
      dtype='object')
Most relevant columns for Mathematics: Index(['G1_y', 'absences_y', 'health', 'Walc', 'Dalc', 'goout', 'freetime',
       'famrel', 'failures', 'studytime'],
      dtype='object')
----------------------------------------------------------------------------------------------------
Training score alpha = 0.1 lasso: 0.6888674941900484
Test score alpha = 0.1 lasso: 0.6911412963788948
Number of features used: 15
Most relevant columns for Portuguese: Index(['G1_x', 'studytime', 'G1_y', 'absences_x', 'Fedu', 'Walc', 'goout',
       'freetime', 'famrel', 'failures'],
      dtype='object')
Most relevant column

Removing also the first grade

Model '5'

In [12]:
features = students_combined.loc[:, ['absences_x', 'Medu', 'Fedu', 'studytime', 
                                                'failures', 'famrel', 'freetime',
                                                'goout', 'Dalc', 'Walc', 'health', 'absences_y']]
target   = students_combined.loc[:, ['G3_x', 'G3_y']]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2)

In [13]:
print('Default number of features: {}'.format(2 * features.shape[1]))
print('{} features for the first predicted value and {} for the second one'.format(features.shape[1], features.shape[1]))
print('=' * 100)

lasso_normal = Lasso(max_iter = 10e6)
lasso_normal = lasso_normal.fit(X_train, y_train)

train_score = lasso_normal.score(X_train, y_train)
test_score  = lasso_normal.score(X_test, y_test)
coefficients_used = np.sum(lasso_normal.coef_ != 0)

print("Training score default lasso: {}".format(train_score))
print("Test score default lasso: {}".format(test_score))
print("Number of features used: {}".format(coefficients_used))

coef1, coef2 = np.array(lasso_normal.coef_)
most_relevant1 = coef1.argsort()[-10:]
most_relevant2 = coef2.argsort()[-10:]
print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))

alpha_values = [0.1, 0.01, 0.001, 2, 5, 10, 15]
for alpha_iter in alpha_values:
    
    lasso = Lasso(alpha = alpha_iter, max_iter = 10e6)
    lasso = lasso.fit(X_train, y_train)

    train_score = lasso.score(X_train, y_train)
    test_score  = lasso.score(X_test, y_test)
    coefficients_used = np.sum(lasso.coef_ != 0)

    print("-" * 100)
    print("Training score alpha = {} lasso: {}".format(alpha_iter, train_score))
    print("Test score alpha = {} lasso: {}".format(alpha_iter, test_score))
    print("Number of features used: {}".format(coefficients_used))

    coef1, coef2 = np.array(lasso.coef_)
    most_relevant1 = coef1.argsort()[-10:]
    most_relevant2 = coef2.argsort()[-10:]
    print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
    print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))

Default number of features: 24
12 features for the first predicted value and 12 for the second one
Training score default lasso: 0.007967825260099853
Test score default lasso: -0.011605557397536041
Number of features used: 3
Most relevant columns for Portuguese: Index(['health', 'Walc', 'Dalc', 'goout', 'freetime', 'famrel', 'failures',
       'studytime', 'Fedu', 'Medu'],
      dtype='object')
Most relevant columns for Mathematics: Index(['health', 'Walc', 'Dalc', 'goout', 'freetime', 'famrel', 'failures',
       'studytime', 'Fedu', 'Medu'],
      dtype='object')
----------------------------------------------------------------------------------------------------
Training score alpha = 0.1 lasso: 0.12363204085345667
Test score alpha = 0.1 lasso: 0.07114238362733785
Number of features used: 17
Most relevant columns for Portuguese: Index(['studytime', 'Medu', 'Fedu', 'absences_x', 'Walc', 'goout', 'famrel',
       'freetime', 'absences_y', 'health'],
      dtype='object')
Most relevant 

So, we have no chance of predicting the final grade without the help of any of the previous grades. 
Let's look at the situation in which we don't have the first grade but we have the second grade.

Model '6'

In [14]:
features = students_combined.loc[:, ['absences_x', 'Medu', 'Fedu', 'studytime', 
                                                'failures', 'famrel', 'freetime',
                                                'goout', 'Dalc', 'Walc', 'health', 'absences_y', 'G2_x', 'G2_y']]
target   = students_combined.loc[:, ['G3_x', 'G3_y']]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2)

In [15]:
print('Default number of features: {}'.format(2 * features.shape[1]))
print('{} features for the first predicted value and {} for the second one'.format(features.shape[1], features.shape[1]))
print('=' * 100)

lasso_normal = Lasso(max_iter = 10e6)
lasso_normal = lasso_normal.fit(X_train, y_train)

train_score = lasso_normal.score(X_train, y_train)
test_score  = lasso_normal.score(X_test, y_test)
coefficients_used = np.sum(lasso_normal.coef_ != 0)

print("Training score default lasso: {}".format(train_score))
print("Test score default lasso: {}".format(test_score))
print("Number of features used: {}".format(coefficients_used))

coef1, coef2 = np.array(lasso_normal.coef_)
most_relevant1 = coef1.argsort()[-10:]
most_relevant2 = coef2.argsort()[-10:]
print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))

alpha_values = [0.1, 0.01, 0.001, 2, 5, 10, 15]
for alpha_iter in alpha_values:
    
    lasso = Lasso(alpha = alpha_iter, max_iter = 10e6)
    lasso = lasso.fit(X_train, y_train)

    train_score = lasso.score(X_train, y_train)
    test_score  = lasso.score(X_test, y_test)
    coefficients_used = np.sum(lasso.coef_ != 0)

    print("-" * 100)
    print("Training score alpha = {} lasso: {}".format(alpha_iter, train_score))
    print("Test score alpha = {} lasso: {}".format(alpha_iter, test_score))
    print("Number of features used: {}".format(coefficients_used))

    coef1, coef2 = np.array(lasso.coef_)
    most_relevant1 = coef1.argsort()[-10:]
    most_relevant2 = coef2.argsort()[-10:]
    print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
    print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))

Default number of features: 28
14 features for the first predicted value and 14 for the second one
Training score default lasso: 0.8083911209399157
Test score default lasso: 0.8435341647448897
Number of features used: 3
Most relevant columns for Portuguese: Index(['G2_x', 'G2_y', 'absences_y', 'health', 'Walc', 'Dalc', 'goout',
       'freetime', 'famrel', 'failures'],
      dtype='object')
Most relevant columns for Mathematics: Index(['G2_y', 'absences_y', 'G2_x', 'health', 'Walc', 'Dalc', 'goout',
       'freetime', 'famrel', 'failures'],
      dtype='object')
----------------------------------------------------------------------------------------------------
Training score alpha = 0.1 lasso: 0.8256308092282985
Test score alpha = 0.1 lasso: 0.8587083136901498
Number of features used: 13
Most relevant columns for Portuguese: Index(['G2_x', 'absences_x', 'Walc', 'Dalc', 'goout', 'freetime', 'famrel',
       'failures', 'studytime', 'Fedu'],
      dtype='object')
Most relevant columns f

It seems that the second grade is more relevant that the first one. So now we try removing one of the second grades but reintroducing a first grade but from the other subject

Model '7'

In [16]:
features = students_combined.loc[:, ['absences_x', 'Medu', 'Fedu', 'studytime', 
                                                'failures', 'famrel', 'freetime',
                                                'goout', 'Dalc', 'Walc', 'health', 'absences_y', 'G2_x', 'G1_y']]
target   = students_combined.loc[:, ['G3_x', 'G3_y']]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2)

In [17]:
print('Default number of features: {}'.format(2 * features.shape[1]))
print('{} features for the first predicted value and {} for the second one'.format(features.shape[1], features.shape[1]))
print('=' * 100)

lasso_normal = Lasso(max_iter = 10e6)
lasso_normal = lasso_normal.fit(X_train, y_train)

train_score = lasso_normal.score(X_train, y_train)
test_score  = lasso_normal.score(X_test, y_test)
coefficients_used = np.sum(lasso_normal.coef_ != 0)

print("Training score default lasso: {}".format(train_score))
print("Test score default lasso: {}".format(test_score))
print("Number of features used: {}".format(coefficients_used))

coef1, coef2 = np.array(lasso_normal.coef_)
most_relevant1 = coef1.argsort()[-10:]
most_relevant2 = coef2.argsort()[-10:]
print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))

alpha_values = [0.1, 0.01, 0.001, 2, 5, 10, 15]
for alpha_iter in alpha_values:
    
    lasso = Lasso(alpha = alpha_iter, max_iter = 10e6)
    lasso = lasso.fit(X_train, y_train)

    train_score = lasso.score(X_train, y_train)
    test_score  = lasso.score(X_test, y_test)
    coefficients_used = np.sum(lasso.coef_ != 0)

    print("-" * 100)
    print("Training score alpha = {} lasso: {}".format(alpha_iter, train_score))
    print("Test score alpha = {} lasso: {}".format(alpha_iter, test_score))
    print("Number of features used: {}".format(coefficients_used))

    coef1, coef2 = np.array(lasso.coef_)
    most_relevant1 = coef1.argsort()[-10:]
    most_relevant2 = coef2.argsort()[-10:]
    print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
    print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))

Default number of features: 28
14 features for the first predicted value and 14 for the second one
Training score default lasso: 0.7043370390386983
Test score default lasso: 0.6660183349381442
Number of features used: 4
Most relevant columns for Portuguese: Index(['G2_x', 'G1_y', 'absences_y', 'health', 'Walc', 'Dalc', 'goout',
       'freetime', 'famrel', 'failures'],
      dtype='object')
Most relevant columns for Mathematics: Index(['G1_y', 'absences_y', 'G2_x', 'health', 'Walc', 'Dalc', 'goout',
       'freetime', 'famrel', 'failures'],
      dtype='object')
----------------------------------------------------------------------------------------------------
Training score alpha = 0.1 lasso: 0.7333941618400319
Test score alpha = 0.1 lasso: 0.667004338457647
Number of features used: 15
Most relevant columns for Portuguese: Index(['G2_x', 'absences_x', 'G1_y', 'health', 'Walc', 'goout', 'freetime',
       'famrel', 'studytime', 'Fedu'],
      dtype='object')
Most relevant columns for 

Model '8'

In [18]:
features = students_combined.loc[:, ['absences_x', 'Medu', 'Fedu', 'studytime', 
                                                'failures', 'famrel', 'freetime',
                                                'goout', 'Dalc', 'Walc', 'health', 'absences_y', 'G1_x', 'G2_y']]
target   = students_combined.loc[:, ['G3_x', 'G3_y']]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2)

In [19]:
print('Default number of features: {}'.format(2 * features.shape[1]))
print('{} features for the first predicted value and {} for the second one'.format(features.shape[1], features.shape[1]))
print('=' * 100)

lasso_normal = Lasso(max_iter = 10e6)
lasso_normal = lasso_normal.fit(X_train, y_train)

train_score = lasso_normal.score(X_train, y_train)
test_score  = lasso_normal.score(X_test, y_test)
coefficients_used = np.sum(lasso_normal.coef_ != 0)

print("Training score default lasso: {}".format(train_score))
print("Test score default lasso: {}".format(test_score))
print("Number of features used: {}".format(coefficients_used))

coef1, coef2 = np.array(lasso_normal.coef_)
most_relevant1 = coef1.argsort()[-10:]
most_relevant2 = coef2.argsort()[-10:]
print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))

alpha_values = [0.1, 0.01, 0.001, 2, 5, 10, 15]
for alpha_iter in alpha_values:
    
    lasso = Lasso(alpha = alpha_iter, max_iter = 10e6)
    lasso = lasso.fit(X_train, y_train)

    train_score = lasso.score(X_train, y_train)
    test_score  = lasso.score(X_test, y_test)
    coefficients_used = np.sum(lasso.coef_ != 0)

    print("-" * 100)
    print("Training score alpha = {} lasso: {}".format(alpha_iter, train_score))
    print("Test score alpha = {} lasso: {}".format(alpha_iter, test_score))
    print("Number of features used: {}".format(coefficients_used))

    coef1, coef2 = np.array(lasso.coef_)
    most_relevant1 = coef1.argsort()[-10:]
    most_relevant2 = coef2.argsort()[-10:]
    print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
    print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))

Default number of features: 28
14 features for the first predicted value and 14 for the second one
Training score default lasso: 0.7713406066096435
Test score default lasso: 0.8001322269175173
Number of features used: 5
Most relevant columns for Portuguese: Index(['G1_x', 'G2_y', 'health', 'Walc', 'Dalc', 'goout', 'freetime', 'famrel',
       'failures', 'studytime'],
      dtype='object')
Most relevant columns for Mathematics: Index(['G2_y', 'absences_y', 'G1_x', 'health', 'Walc', 'Dalc', 'goout',
       'freetime', 'famrel', 'failures'],
      dtype='object')
----------------------------------------------------------------------------------------------------
Training score alpha = 0.1 lasso: 0.7966479891020227
Test score alpha = 0.1 lasso: 0.8102655169609657
Number of features used: 17
Most relevant columns for Portuguese: Index(['G1_x', 'famrel', 'G2_y', 'absences_x', 'studytime', 'Walc', 'goout',
       'freetime', 'Fedu', 'Medu'],
      dtype='object')
Most relevant columns for Ma

By looking at the two situations we find that we are able to predict the final grades for Portuguese and Mathematics when we only have the second grade for Mathematics and the first grade for Portuguese than when the first grade in Mathematics and the second grade in Portuguese is known.

Model '9'

In [20]:
features = students_combined.loc[:, ['absences_x', 'Medu', 'Fedu', 'studytime', 
                                                'failures', 'famrel', 'freetime',
                                                'goout', 'Dalc', 'Walc', 'health', 'absences_y', 'G1_x', 'G2_y', 'G2_x']]
target   = students_combined.loc[:, ['G3_x', 'G3_y']]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2)

In [21]:
print('Default number of features: {}'.format(2 * features.shape[1]))
print('{} features for the first predicted value and {} for the second one'.format(features.shape[1], features.shape[1]))
print('=' * 100)

lasso_normal = Lasso(max_iter = 10e6)
lasso_normal = lasso_normal.fit(X_train, y_train)

train_score = lasso_normal.score(X_train, y_train)
test_score  = lasso_normal.score(X_test, y_test)
coefficients_used = np.sum(lasso_normal.coef_ != 0)

print("Training score default lasso: {}".format(train_score))
print("Test score default lasso: {}".format(test_score))
print("Number of features used: {}".format(coefficients_used))

coef1, coef2 = np.array(lasso_normal.coef_)
most_relevant1 = coef1.argsort()[-10:]
most_relevant2 = coef2.argsort()[-10:]
print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))

alpha_values = [0.1, 0.01, 0.001, 2, 5, 10, 15]
for alpha_iter in alpha_values:
    
    lasso = Lasso(alpha = alpha_iter, max_iter = 10e6)
    lasso = lasso.fit(X_train, y_train)

    train_score = lasso.score(X_train, y_train)
    test_score  = lasso.score(X_test, y_test)
    coefficients_used = np.sum(lasso.coef_ != 0)

    print("-" * 100)
    print("Training score alpha = {} lasso: {}".format(alpha_iter, train_score))
    print("Test score alpha = {} lasso: {}".format(alpha_iter, test_score))
    print("Number of features used: {}".format(coefficients_used))

    coef1, coef2 = np.array(lasso.coef_)
    most_relevant1 = coef1.argsort()[-10:]
    most_relevant2 = coef2.argsort()[-10:]
    print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
    print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))

Default number of features: 30
15 features for the first predicted value and 15 for the second one
Training score default lasso: 0.8033740984156065
Test score default lasso: 0.8602270078171199
Number of features used: 4
Most relevant columns for Portuguese: Index(['G2_x', 'G1_x', 'G2_y', 'absences_y', 'health', 'Walc', 'Dalc', 'goout',
       'freetime', 'famrel'],
      dtype='object')
Most relevant columns for Mathematics: Index(['G2_y', 'absences_y', 'G2_x', 'G1_x', 'health', 'Walc', 'Dalc', 'goout',
       'freetime', 'famrel'],
      dtype='object')
----------------------------------------------------------------------------------------------------
Training score alpha = 0.1 lasso: 0.8211639471449957
Test score alpha = 0.1 lasso: 0.874688300690012
Number of features used: 17
Most relevant columns for Portuguese: Index(['G2_x', 'G1_x', 'absences_x', 'Walc', 'Dalc', 'goout', 'freetime',
       'famrel', 'failures', 'studytime'],
      dtype='object')
Most relevant columns for Mathem

Model '10'

In [39]:
features = students_combined.loc[:, ['absences_x', 'Medu', 'Fedu', 'studytime', 
                                                'failures', 'famrel', 'freetime',
                                                'goout', 'Dalc', 'Walc', 'health', 'absences_y', 'G1_y', 'G2_y']]
target   = students_combined.loc[:, ['G3_x', 'G3_y']]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2)

In [40]:
print('Default number of features: {}'.format(2 * features.shape[1]))
print('{} features for the first predicted value and {} for the second one'.format(features.shape[1], features.shape[1]))
print('=' * 100)

lasso_normal = Lasso(max_iter = 10e6)
lasso_normal = lasso_normal.fit(X_train, y_train)

train_score = lasso_normal.score(X_train, y_train)
test_score  = lasso_normal.score(X_test, y_test)
coefficients_used = np.sum(lasso_normal.coef_ != 0)

print("Training score default lasso: {}".format(train_score))
print("Test score default lasso: {}".format(test_score))
print("Number of features used: {}".format(coefficients_used))

coef1, coef2 = np.array(lasso_normal.coef_)
most_relevant1 = coef1.argsort()[-10:]
most_relevant2 = coef2.argsort()[-10:]
print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))


alpha_values = [0.1, 0.01, 0.001, 2, 5, 10, 15]
for alpha_iter in alpha_values:
    
    lasso = Lasso(alpha = alpha_iter, max_iter = 10e6)
    lasso = lasso.fit(X_train, y_train)

    train_score = lasso.score(X_train, y_train)
    test_score  = lasso.score(X_test, y_test)
    coefficients_used = np.sum(lasso.coef_ != 0)

    print("-" * 100)
    print("Training score alpha = {} lasso: {}".format(alpha_iter, train_score))
    print("Test score alpha = {} lasso: {}".format(alpha_iter, test_score))
    print("Number of features used: {}".format(coefficients_used))

    coef1, coef2 = np.array(lasso.coef_)
    most_relevant1 = coef1.argsort()[-10:]
    most_relevant2 = coef2.argsort()[-10:]
    print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
    print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))

Default number of features: 28
14 features for the first predicted value and 14 for the second one
Training score default lasso: 0.6698105147584451
Test score default lasso: 0.6413837256401995
Number of features used: 5
Most relevant columns for Portuguese: Index(['G1_y', 'G2_y', 'health', 'Walc', 'Dalc', 'goout', 'freetime', 'famrel',
       'failures', 'studytime'],
      dtype='object')
Most relevant columns for Mathematics: Index(['G2_y', 'G1_y', 'absences_y', 'health', 'Walc', 'Dalc', 'goout',
       'freetime', 'famrel', 'failures'],
      dtype='object')
----------------------------------------------------------------------------------------------------
Training score alpha = 0.1 lasso: 0.7163527259385818
Test score alpha = 0.1 lasso: 0.6688180043378334
Number of features used: 19
Most relevant columns for Portuguese: Index(['G1_y', 'studytime', 'Medu', 'G2_y', 'absences_x', 'Walc', 'goout',
       'freetime', 'famrel', 'Fedu'],
      dtype='object')
Most relevant columns for Ma

Model '11'

In [35]:
features = students_combined.loc[:, ['absences_x', 'Medu', 'Fedu', 'studytime', 
                                                'failures', 'famrel', 'freetime',
                                                'goout', 'Dalc', 'Walc', 'health', 'absences_y', 'G1_x', 'G2_x']]
target   = students_combined.loc[:, ['G3_x', 'G3_y']]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2)

In [38]:
print('Default number of features: {}'.format(2 * features.shape[1]))
print('{} features for the first predicted value and {} for the second one'.format(features.shape[1], features.shape[1]))
print('=' * 100)

lasso_normal = Lasso(max_iter = 10e6)
lasso_normal = lasso_normal.fit(X_train, y_train)

train_score = lasso_normal.score(X_train, y_train)
test_score  = lasso_normal.score(X_test, y_test)
coefficients_used = np.sum(lasso_normal.coef_ != 0)

print("Training score default lasso: {}".format(train_score))
print("Test score default lasso: {}".format(test_score))
print("Number of features used: {}".format(coefficients_used))

coef1, coef2 = np.array(lasso_normal.coef_)
most_relevant1 = coef1.argsort()[-10:]
most_relevant2 = coef2.argsort()[-10:]
print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))


alpha_values = [0.1, 0.01, 0.001, 2, 5, 10, 15]
for alpha_iter in alpha_values:
    
    lasso = Lasso(alpha = alpha_iter, max_iter = 10e6)
    lasso = lasso.fit(X_train, y_train)

    train_score = lasso.score(X_train, y_train)
    test_score  = lasso.score(X_test, y_test)
    coefficients_used = np.sum(lasso.coef_ != 0)

    print("-" * 100)
    print("Training score alpha = {} lasso: {}".format(alpha_iter, train_score))
    print("Test score alpha = {} lasso: {}".format(alpha_iter, test_score))
    print("Number of features used: {}".format(coefficients_used))

    coef1, coef2 = np.array(lasso.coef_)
    most_relevant1 = coef1.argsort()[-10:]
    most_relevant2 = coef2.argsort()[-10:]
    print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
    print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))

Default number of features: 28
14 features for the first predicted value and 14 for the second one
Training score default lasso: 0.47753119404262495
Test score default lasso: 0.3268349968499021
Number of features used: 6
Most relevant columns for Portuguese: Index(['G2_x', 'G1_x', 'absences_y', 'health', 'Walc', 'Dalc', 'goout',
       'freetime', 'famrel', 'failures'],
      dtype='object')
Most relevant columns for Mathematics: Index(['G1_x', 'G2_x', 'absences_y', 'health', 'Walc', 'Dalc', 'goout',
       'freetime', 'famrel', 'failures'],
      dtype='object')
----------------------------------------------------------------------------------------------------
Training score alpha = 0.1 lasso: 0.5183606037755887
Test score alpha = 0.1 lasso: 0.31655478028295214
Number of features used: 19
Most relevant columns for Portuguese: Index(['G2_x', 'G1_x', 'absences_x', 'studytime', 'health', 'Walc', 'goout',
       'freetime', 'famrel', 'Fedu'],
      dtype='object')
Most relevant columns f

Model '12'

In [27]:
features = students_combined.loc[:, ['absences_x', 'Medu', 'Fedu', 'studytime', 
                                                'failures', 'famrel', 'freetime',
                                                'goout', 'Dalc', 'Walc', 'health', 'absences_y', 'G2_x']]
target   = students_combined.loc[:, ['G3_x', 'G3_y']]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2)

In [28]:
print('Default number of features: {}'.format(2 * features.shape[1]))
print('{} features for the first predicted value and {} for the second one'.format(features.shape[1], features.shape[1]))
print('=' * 100)

lasso_normal = Lasso(max_iter = 10e6)
lasso_normal = lasso_normal.fit(X_train, y_train)

train_score = lasso_normal.score(X_train, y_train)
test_score  = lasso_normal.score(X_test, y_test)
coefficients_used = np.sum(lasso_normal.coef_ != 0)

print("Training score default lasso: {}".format(train_score))
print("Test score default lasso: {}".format(test_score))
print("Number of features used: {}".format(coefficients_used))

coef1, coef2 = np.array(lasso_normal.coef_)
most_relevant1 = coef1.argsort()[-10:]
most_relevant2 = coef2.argsort()[-10:]
print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))


alpha_values = [0.1, 0.01, 0.001, 2, 5, 10, 15]
for alpha_iter in alpha_values:
    
    lasso = Lasso(alpha = alpha_iter, max_iter = 10e6)
    lasso = lasso.fit(X_train, y_train)

    train_score = lasso.score(X_train, y_train)
    test_score  = lasso.score(X_test, y_test)
    coefficients_used = np.sum(lasso.coef_ != 0)

    print("-" * 100)
    print("Training score alpha = {} lasso: {}".format(alpha_iter, train_score))
    print("Test score alpha = {} lasso: {}".format(alpha_iter, test_score))
    print("Number of features used: {}".format(coefficients_used))

    coef1, coef2 = np.array(lasso.coef_)
    most_relevant1 = coef1.argsort()[-10:]
    most_relevant2 = coef2.argsort()[-10:]
    print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
    print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))

Default number of features: 26
13 features for the first predicted value and 13 for the second one
Training score default lasso: 0.4561180328429293
Test score default lasso: 0.3800153650526898
Number of features used: 4
Most relevant columns for Portuguese: Index(['G2_x', 'absences_y', 'health', 'Walc', 'Dalc', 'goout', 'freetime',
       'famrel', 'failures', 'studytime'],
      dtype='object')
Most relevant columns for Mathematics: Index(['G2_x', 'absences_y', 'health', 'Walc', 'Dalc', 'goout', 'freetime',
       'famrel', 'failures', 'studytime'],
      dtype='object')
----------------------------------------------------------------------------------------------------
Training score alpha = 0.1 lasso: 0.492778293034061
Test score alpha = 0.1 lasso: 0.4112325369124292
Number of features used: 16
Most relevant columns for Portuguese: Index(['G2_x', 'absences_x', 'Walc', 'goout', 'famrel', 'failures',
       'studytime', 'Fedu', 'Medu', 'health'],
      dtype='object')
Most relevant co

Model '13'

In [29]:
features = students_combined.loc[:, ['absences_x', 'Medu', 'Fedu', 'studytime', 
                                                'failures', 'famrel', 'freetime',
                                                'goout', 'Dalc', 'Walc', 'health', 'absences_y', 'G2_y']]
target   = students_combined.loc[:, ['G3_x', 'G3_y']]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2)

In [30]:
print('Default number of features: {}'.format(2 * features.shape[1]))
print('{} features for the first predicted value and {} for the second one'.format(features.shape[1], features.shape[1]))
print('=' * 100)

lasso_normal = Lasso(max_iter = 10e6)
lasso_normal = lasso_normal.fit(X_train, y_train)

train_score = lasso_normal.score(X_train, y_train)
test_score  = lasso_normal.score(X_test, y_test)
coefficients_used = np.sum(lasso_normal.coef_ != 0)

print("Training score default lasso: {}".format(train_score))
print("Test score default lasso: {}".format(test_score))
print("Number of features used: {}".format(coefficients_used))

coef1, coef2 = np.array(lasso_normal.coef_)
most_relevant1 = coef1.argsort()[-10:]
most_relevant2 = coef2.argsort()[-10:]
print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))


alpha_values = [0.1, 0.01, 0.001, 2, 5, 10, 15]
for alpha_iter in alpha_values:
    
    lasso = Lasso(alpha = alpha_iter, max_iter = 10e6)
    lasso = lasso.fit(X_train, y_train)

    train_score = lasso.score(X_train, y_train)
    test_score  = lasso.score(X_test, y_test)
    coefficients_used = np.sum(lasso.coef_ != 0)

    print("-" * 100)
    print("Training score alpha = {} lasso: {}".format(alpha_iter, train_score))
    print("Test score alpha = {} lasso: {}".format(alpha_iter, test_score))
    print("Number of features used: {}".format(coefficients_used))

    coef1, coef2 = np.array(lasso.coef_)
    most_relevant1 = coef1.argsort()[-10:]
    most_relevant2 = coef2.argsort()[-10:]
    print("Most relevant columns for Portuguese: {}".format(features.columns[most_relevant1[::-1]]))
    print("Most relevant columns for Mathematics: {}".format(features.columns[most_relevant2[::-1]]))

Default number of features: 26
13 features for the first predicted value and 13 for the second one
Training score default lasso: 0.6352816058935932
Test score default lasso: 0.7356939084753624
Number of features used: 4
Most relevant columns for Portuguese: Index(['G2_y', 'health', 'Walc', 'Dalc', 'goout', 'freetime', 'famrel',
       'failures', 'studytime', 'Fedu'],
      dtype='object')
Most relevant columns for Mathematics: Index(['G2_y', 'absences_y', 'health', 'Walc', 'Dalc', 'goout', 'freetime',
       'famrel', 'failures', 'studytime'],
      dtype='object')
----------------------------------------------------------------------------------------------------
Training score alpha = 0.1 lasso: 0.6905930755877008
Test score alpha = 0.1 lasso: 0.7503915210142388
Number of features used: 18
Most relevant columns for Portuguese: Index(['studytime', 'G2_y', 'Medu', 'goout', 'absences_x', 'Walc', 'freetime',
       'famrel', 'Fedu', 'absences_y'],
      dtype='object')
Most relevant col

Looking at the previous statistics we can say the following:
- having the second Mathematics grade is more relevant than having the second Portuguese grade.
- if we have both Portuguese grades and the second Mathematics grade we don't have any significant improvement compared to the case in which we only have the second Mathematics grade and a Portuguese grade.
- if we have only the Mathematics grades the overall prediction score declines even though not drastically compared to the situation in which we only have only the Portuguese grades case in which the prediction score is almost a half (of that from the Mathematics only scenario).